# 🎴 Pokemon Card Detection - YOLO11 Training Notebook

## What You'll Learn
This notebook will teach you how to train a **YOLO11** object detection model to find Pokemon cards in images. By the end, you'll have:

1. A trained model that draws bounding boxes around Pokemon cards
2. Understanding of how YOLO training works
3. A model ready to use with PriceLens

## Dataset: Cleveland's 5K Real-World Images
We're using the **Pokemon Card Identification** dataset from Roboflow:
- **5,058 real-world photos** (not clean scans!)
- Cards held in hands, on desks, in binders
- Various lighting conditions and angles
- Multiple cards per image scenarios
- License: CC BY 4.0

This matches what your webcam will actually see - much better than training on clean card scans!

## Detection vs Identification
- **Detection** (this notebook): Finding WHERE cards are in an image → Bounding boxes
- **Identification** (separate step): Figuring out WHICH card it is → "Base Set Charizard"

Think of it like this: Detection is like a security camera finding "there's a person at coordinates (x,y)". Identification is recognizing "that's Bob from accounting".

## Why YOLO11?
YOLO (You Only Look Once) is a family of fast object detection models. YOLO11 is the latest version with:
- **Speed**: 30+ FPS on modern GPUs
- **Accuracy**: State-of-the-art detection performance
- **Simplicity**: Easy to train and deploy

We'll use **YOLO11n** (nano) - the smallest and fastest variant, perfect for real-time applications.


## Section 1: Environment Setup

First, let's check if we're running in Google Colab or locally, then install the required packages.


In [ ]:
import sys
print(f"Python path: {sys.executable}")
print(f"Python version: {sys.version}")

import cv2, torch, ultralytics
print(f"\n✓ OpenCV: {cv2.__version__}")
print(f"✓ PyTorch: {torch.__version__}")
print(f"✓ YOLO: {ultralytics.__version__}")
print(f"✓ CUDA Available: {torch.cuda.is_available()}")

In [ ]:
# ============================================================
# Section 1: Environment Setup
# ============================================================

import sys
import os

# Detect environment
IN_COLAB = 'google.colab' in sys.modules
print(f"🖥️  Environment: {'Google Colab' if IN_COLAB else 'Local'}")

# Install dependencies
if IN_COLAB:
    print("📦 Installing dependencies for Colab...")
    %pip install -q ultralytics roboflow opencv-python matplotlib pandas
else:
    print("📦 Using local environment - ensure requirements.txt is installed")
    # For local: pip install ultralytics roboflow opencv-python matplotlib pandas


In [ ]:
# Verify GPU availability
import torch

print("\n🔧 Hardware Check:")
print(f"   PyTorch version: {torch.__version__}")
print(f"   CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    DEVICE = 0  # Use first GPU
else:
    print("   ⚠️  No GPU detected - training will be slower on CPU")
    DEVICE = 'cpu'

print(f"\n✅ Will use device: {DEVICE}")


## Section 2: Download the Dataset

We'll download the **Cleveland Pokemon Card** dataset from Roboflow - 5,058 real-world photos!

### Why This Dataset?
| Feature | Clean Scans (Bad) | Cleveland Dataset (Good) |
|---------|-------------------|--------------------------|
| Images | ~400 | **5,058** |
| Type | Card fills entire frame | Real photos - hands, desks, binders |
| Angles | Perfect alignment | Various rotations |
| Lighting | Studio lighting | Real-world conditions |
| Cards/Image | Always 1 | Often multiple |

### YOLO Annotation Format
Each image has a `.txt` file with bounding boxes:
```
class_id  x_center  y_center  width  height
0         0.45      0.52      0.30   0.42
```
All values are normalized (0-1) so they work at any resolution.

### You'll Need a Roboflow API Key (Free!)
1. Go to https://app.roboflow.com (create free account)
2. Click your profile → Settings → API Key
3. Copy the key and paste it below


In [ ]:
# ============================================================
# Section 2: Download the Dataset
# ============================================================

from roboflow import Roboflow
from pathlib import Path
import shutil

# ⚠️ PASTE YOUR ROBOFLOW API KEY HERE ⚠️
# Get it free at: https://app.roboflow.com/settings/api
ROBOFLOW_API_KEY = os.getenv("ROBOFLOW_API_KEY")

# Determine base directory (handle both notebook and script execution)
if IN_COLAB:
    BASE_DIR = Path.cwd()
else:
    # Assume notebook is in notebooks/ folder, so go up one level to project root
    BASE_DIR = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()

# Dataset will be saved in the data folder
DATA_DIR = BASE_DIR / "data" / "datasets"
DATASET_NAME = "pokemon-card-detection"
DATASET_PATH = None

# Check if dataset already exists in data folder
expected_path = DATA_DIR / DATASET_NAME

if expected_path.exists() and (expected_path / "data.yaml").exists():
    DATASET_PATH = str(expected_path)
    
    # Validate it's the correct dataset (Cleveland 5K, not a small dataset)
    with open(expected_path / "data.yaml", 'r') as f:
        import yaml
        data_config = yaml.safe_load(f)
        
    # Check if it's the Cleveland dataset
    if 'cleveland' in data_config.get('roboflow', {}).get('workspace', '').lower():
        # Count images to verify
        train_images = list((expected_path / "train" / "images").glob("*.jpg"))
        if len(train_images) > 1000:  # Should have ~4K training images
            print(f"✅ Found Cleveland dataset with {len(train_images):,} training images")
            print(f"   Location: {DATASET_PATH}")
            print("   Skipping download - using cached version")
        else:
            print(f"⚠️  Found dataset but only {len(train_images)} images (expected 4000+)")
            print("   Removing and will re-download...")
            shutil.rmtree(expected_path)
            expected_path = None
    else:
        print("⚠️  Found wrong dataset (not Cleveland). Removing...")
        shutil.rmtree(expected_path)
        expected_path = None

# Download if not found or was wrong dataset
if not expected_path or not expected_path.exists():
    if ROBOFLOW_API_KEY == "YOUR_API_KEY_HERE" or ROBOFLOW_API_KEY is None:
        print("❌ ERROR: Please set your Roboflow API key!")
        print("")
        print("   1. Go to https://app.roboflow.com (create free account)")
        print("   2. Click profile icon → Settings → API Key")
        print("   3. Set ROBOFLOW_API_KEY environment variable")
        print("")
        print("   Example: export ROBOFLOW_API_KEY='abc123xyz456'")
        DATASET_PATH = None
    else:
        print("📥 Downloading Cleveland Pokemon Card Dataset...")
        print("   ⚠️  IMPORTANT: This is the LARGE dataset (5,058 images)")
        print("   Source: https://universe.roboflow.com/cleveland-nahux/pokemon-card-identification")
        print("   Workspace: cleveland-nahux")
        print("   Project: pokemon-card-identification") 
        print("   Version: 5")
        print("   License: CC BY 4.0")
        print("")
        print("   Download size: ~2-3 GB")
        print("   This may take 5-10 minutes...")
        print("")
        
        try:
            # Create data directory if it doesn't exist
            DATA_DIR.mkdir(parents=True, exist_ok=True)
            
            # Initialize Roboflow
            rf = Roboflow(api_key=ROBOFLOW_API_KEY)
            
            # Download the Cleveland dataset (5,058 real-world images)
            # ⚠️ Make sure to use the correct workspace and project!
            print("   Loading workspace 'cleveland-nahux'...")
            workspace = rf.workspace("cleveland-nahux")
            
            print("   Loading project 'pokemon-card-identification'...")
            project = workspace.project("pokemon-card-identification")
            
            print("   Downloading version 5 in YOLOv11 format...")
            # Download in YOLOv11 format
            # NOTE: Roboflow ignores the location parameter and downloads to CWD
            dataset = project.version(5).download("yolov11")
            
            # Get actual download location
            download_path = Path(dataset.location)
            print(f"   ✓ Downloaded to: {download_path}")
            
            # Move to organized location in data/datasets/
            expected_path = DATA_DIR / DATASET_NAME
            
            if download_path != expected_path:
                print(f"   Moving to: {expected_path}")
                
                # Remove destination if it exists
                if expected_path.exists():
                    shutil.rmtree(expected_path)
                
                # Move dataset
                shutil.move(str(download_path), str(expected_path))
                DATASET_PATH = str(expected_path)
                print(f"   ✓ Dataset organized")
            else:
                DATASET_PATH = str(download_path)
            
            # Verify dataset
            train_images = list((expected_path / "train" / "images").glob("*.jpg"))
            print(f"\n✅ Dataset ready!")
            print(f"   Location: {DATASET_PATH}")
            print(f"   Training images: {len(train_images):,}")
            
            if len(train_images) < 1000:
                print(f"\n   ⚠️  WARNING: Only {len(train_images)} training images found!")
                print("   Expected ~4,000 images from Cleveland dataset")
                print("   You may have downloaded the wrong dataset/version")
            
        except Exception as e:
            print(f"\n❌ Download failed: {e}")
            print("")
            print("   Troubleshooting:")
            print("   - Check your API key is correct")
            print("   - Verify workspace name: cleveland-nahux")
            print("   - Verify project name: pokemon-card-identification")
            print("   - Try version 4 if version 5 fails")
            print("   - Check internet connection")
            DATASET_PATH = None


In [ ]:
# ============================================================
# Section 2b: Analyze Dataset Statistics
# ============================================================

import yaml

def analyze_dataset(path):
    """Analyze dataset structure and statistics"""
    print("📊 Dataset Analysis")
    print("=" * 60)
    
    stats = {}
    total_images = 0
    total_boxes = 0
    
    for split in ['train', 'valid', 'val', 'test']:
        images_dir = os.path.join(path, split, 'images')
        labels_dir = os.path.join(path, split, 'labels')
        
        if not os.path.exists(images_dir):
            continue
        
        # Count images
        images = [f for f in os.listdir(images_dir) 
                 if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
        
        # Count bounding boxes and analyze
        split_boxes = 0
        boxes_per_image = []
        
        for img in images:
            label_file = os.path.splitext(img)[0] + '.txt'
            label_path = os.path.join(labels_dir, label_file)
            
            if os.path.exists(label_path):
                with open(label_path, 'r') as f:
                    num_boxes = len([l for l in f.readlines() if l.strip()])
                    split_boxes += num_boxes
                    boxes_per_image.append(num_boxes)
        
        avg_boxes = sum(boxes_per_image) / len(boxes_per_image) if boxes_per_image else 0
        
        stats[split] = {
            'images': len(images),
            'boxes': split_boxes,
            'avg_boxes': avg_boxes
        }
        
        total_images += len(images)
        total_boxes += split_boxes
        
        print(f"\n{split.upper():6s}:")
        print(f"   Images: {len(images):,}")
        print(f"   Bounding boxes: {split_boxes:,}")
        print(f"   Avg cards/image: {avg_boxes:.2f}")
    
    print("\n" + "=" * 60)
    print(f"TOTAL: {total_images:,} images, {total_boxes:,} bounding boxes")
    
    # Load and show data.yaml
    data_yaml_path = os.path.join(path, 'data.yaml')
    if os.path.exists(data_yaml_path):
        with open(data_yaml_path, 'r') as f:
            data_config = yaml.safe_load(f)
        print(f"\nClasses: {data_config.get('names', 'N/A')}")
        print(f"Number of classes: {data_config.get('nc', 'N/A')}")
    
    return total_images, stats

if DATASET_PATH:
    num_images, dataset_stats = analyze_dataset(DATASET_PATH)
else:
    print("❌ Cannot analyze - dataset not downloaded")


## Section 3: Visualize Training Data

**Always look at your data before training!** This helps you:
1. Verify annotations are correct
2. Understand what the model will learn
3. Spot potential issues (bad labels, poor image quality)

Let's see some samples with their bounding boxes drawn.


In [ ]:
# ============================================================
# Section 3: Visualize Training Data
# ============================================================

import cv2
import matplotlib.pyplot as plt
import random

def visualize_samples(dataset_path, num_samples=6):
    """Show random samples from training set with bounding boxes"""
    
    # Find images directory
    train_images = os.path.join(dataset_path, 'train', 'images')
    train_labels = os.path.join(dataset_path, 'train', 'labels')
    
    if not os.path.exists(train_images):
        print(f"❌ Training images not found at {train_images}")
        return
    
    # Get all images
    all_images = [f for f in os.listdir(train_images) 
                  if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    if len(all_images) == 0:
        print("❌ No images found in training set")
        return
    
    # Select random samples
    samples = random.sample(all_images, min(num_samples, len(all_images)))
    
    # Load class names
    data_yaml_path = os.path.join(dataset_path, 'data.yaml')
    class_names = ['card']  # Default
    if os.path.exists(data_yaml_path):
        with open(data_yaml_path, 'r') as f:
            data = yaml.safe_load(f)
            class_names = data.get('names', ['card'])
    
    # Create grid
    rows = 2
    cols = 3
    fig, axes = plt.subplots(rows, cols, figsize=(15, 10))
    axes = axes.flatten()
    
    for idx, img_name in enumerate(samples):
        img_path = os.path.join(train_images, img_name)
        img = cv2.imread(img_path)
        
        if img is None:
            continue
            
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w = img.shape[:2]
        
        # Load and draw bounding boxes
        label_name = os.path.splitext(img_name)[0] + '.txt'
        label_path = os.path.join(train_labels, label_name)
        num_boxes = 0
        
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        cls_id = int(parts[0])
                        xc, yc, bw, bh = map(float, parts[1:5])
                        
                        # Convert YOLO to pixel coords
                        x1 = int((xc - bw/2) * w)
                        y1 = int((yc - bh/2) * h)
                        x2 = int((xc + bw/2) * w)
                        y2 = int((yc + bh/2) * h)
                        
                        # Draw box
                        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        
                        # Add label
                        label = class_names[cls_id] if cls_id < len(class_names) else f"cls_{cls_id}"
                        cv2.putText(img, label, (x1, y1-5), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                        num_boxes += 1
        
        axes[idx].imshow(img)
        axes[idx].set_title(f"{num_boxes} card(s)", fontsize=10)
        axes[idx].axis('off')
    
    plt.suptitle(f'Cleveland Dataset Samples ({len(all_images):,} training images)', 
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print(f"✅ Showing {len(samples)} random samples from {len(all_images):,} training images")

# Visualize samples
if DATASET_PATH:
    visualize_samples(DATASET_PATH)
else:
    print("❌ Cannot visualize - dataset not downloaded")


## Section 4: Configure Training

Before we train, let's understand the key settings:

### Key Hyperparameters
- **epochs**: How many times the model sees ALL images (like re-reading a textbook)
- **batch**: Images processed together - limited by GPU memory (reduce if you get OOM errors)
- **imgsz**: Input image size - larger means more detail but slower training
- **patience**: Stop early if no improvement for N epochs (saves time!)

### Transfer Learning
We start from **pre-trained weights** (yolo11n.pt) trained on COCO dataset (80 object types). This gives the model a "head start" - it already knows edges, shapes, textures. We just teach it what a Pokemon card looks like.

Starting from scratch would need 10x more data and time!


In [ ]:
# ============================================================
# Section 4: Configure Training
# ============================================================

from ultralytics import YOLO

# Path to data.yaml (from downloaded dataset)
DATA_YAML = os.path.join(DATASET_PATH, 'data.yaml') if DATASET_PATH else None

# Training configuration - optimized for 10K dataset with GPU
CONFIG = {
    # Core settings
    'epochs': 100,           # Full training
    'batch': 16,             # Good for 8GB GPU
    'imgsz': 640,            # Standard YOLO input size
    'patience': 20,          # Early stopping patience
    'device': 0,             # GPU
    
    # Augmentation (less aggressive since dataset already has variety)
    'degrees': 15,           # Rotation range
    'translate': 0.1,        # Translation
    'scale': 0.3,            # Scale variation
    'fliplr': 0.5,           # Horizontal flip probability
    'mosaic': 1.0,           # Mosaic augmentation
    
    # Optimizer
    'optimizer': 'AdamW',
    'lr0': 0.001,            # Initial learning rate
    'lrf': 0.01,             # Final LR factor
    
    # Performance (adjusted for cuDNN stability)
    'workers': 0,            # Fix cuDNN stream issues
    'cache': True,           # RAM cache instead of disk
    'amp': False,            # Disable AMP to avoid cuDNN issues
}

print("⚙️  Training Configuration:")
print("=" * 60)
print(f"   {'epochs':12s}: {CONFIG['epochs']}")
print(f"   {'batch':12s}: {CONFIG['batch']}")
print(f"   {'imgsz':12s}: {CONFIG['imgsz']}")
print(f"   {'patience':12s}: {CONFIG['patience']}")
print(f"   {'device':12s}: {CONFIG['device']}")
print(f"   {'optimizer':12s}: {CONFIG['optimizer']}")
print(f"   {'workers':12s}: {CONFIG['workers']} (0 to avoid cuDNN issues)")
print(f"   {'amp':12s}: {CONFIG['amp']} (disabled for stability)")
print("")
print("   🎮 Training on GPU with cuDNN stability fixes")
print("   ⏱️  Estimated time: 2-3 hours for 100 epochs")
print("=" * 60)

# Verify data.yaml
if DATA_YAML and os.path.exists(DATA_YAML):
    print(f"\n📄 Dataset config: {DATA_YAML}")
    print("-" * 60)
    with open(DATA_YAML, 'r') as f:
        print(f.read())
    print("-" * 60)
    print("✅ Ready to train!")
else:
    print("\n❌ data.yaml not found - download the dataset first!")


## Section 5: Train the Model

### ⚠️ CUDA "Illegal Instruction" Error?

If GPU training crashes with this error, it's a **PyTorch/CUDA driver mismatch**.

**Quick Fix:** Set `FORCE_CPU = True` in the code cell below

**Proper Fix:** Reinstall PyTorch in terminal:
```bash
pip uninstall torch torchvision torchaudio -y
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
```
Then restart the kernel and re-run all cells.

### ✅ Checkpoint Resume Support

**The training cell now supports automatic resume!**

- YOLO saves checkpoints every epoch as `last.pt`
- If training stops (crash, interrupt, early exit), just **run the cell again**
- It will automatically detect the checkpoint and resume from the last epoch
- Your progress is never lost!

**How it works:**
1. First run: Trains from epoch 1
2. If interrupted at epoch 10: Next run starts at epoch 11
3. Continues until epoch 100 (or early stopping triggers)

### Training Settings
| Setting | GPU | CPU |
|---------|-----|-----|
| Epochs | 100 | 50 |
| Batch | 16 | 8 |
| Cache | RAM | off |
| Time | ~2-3 hours | ~4-6 hours |

### Check Training Status Before Starting

Run the cell below to see if you have an existing checkpoint:

In [ ]:
# ============================================================
# Section 5A: HYBRID TRAINING - Local GPU with Colab Fallback
# ============================================================

"""
🎯 THIS CELL DOES BOTH:
1. Tries LOCAL GPU first with CUDA error fix
2. If it fails 2x, tells you how to switch to Colab
3. Your epoch 8 checkpoint is preserved either way!
"""

import os
import sys

# ⚠️ CUDA FIX: Prevents "illegal instruction" errors
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
print("🔧 Applied CUDA_LAUNCH_BLOCKING=1 fix")

# Initialize YOLO11 nano
print("\n🚀 Initializing YOLO11n model...")

# Output directory
PROJECT_DIR = '../models/training_runs'
RUN_NAME = 'yolo11n_cleveland_notebook'
CHECKPOINT_PATH = f'{PROJECT_DIR}/{RUN_NAME}/weights/last.pt'
RESUME_TRAINING = os.path.exists(CHECKPOINT_PATH)

# Check current environment
IN_COLAB = 'google.colab' in sys.modules

if RESUME_TRAINING:
    print(f"✅ Found checkpoint: {CHECKPOINT_PATH}")
    
    # Check what epoch we're at
    results_csv = f'{PROJECT_DIR}/{RUN_NAME}/results.csv'
    if os.path.exists(results_csv):
        import pandas as pd
        df = pd.read_csv(results_csv)
        last_epoch = len(df)
        print(f"   📈 Resuming from epoch {last_epoch} (will start epoch {last_epoch + 1})")
    else:
        print("   📈 Resuming from checkpoint (epoch unknown)")
    
    model = YOLO(CHECKPOINT_PATH)
else:
    print("   Starting fresh training from pretrained weights...")
    model = YOLO('yolo11n.pt')

print(f"\n📁 Results: {PROJECT_DIR}/{RUN_NAME}/")
print(f"🖥️  Environment: {'Google Colab' if IN_COLAB else 'Local Machine'}")
print("=" * 60)

if IN_COLAB:
    print("🌐 COLAB MODE DETECTED")
    print("   Using Colab GPU (T4 or better)")
    print("   ⏱️  Estimated: 4-6 hours for remaining epochs")
else:
    print("🎮 LOCAL GPU MODE")
    print("   Using: NVIDIA GeForce RTX 4070 Laptop GPU")
    print("   ⏱️  Estimated: 2-3 hours for remaining epochs")
    print("   🔧 CUDA fix applied: CUDA_LAUNCH_BLOCKING=1")

print("=" * 60)

# Training attempt with error handling
try:
    results = model.train(
        data=DATA_YAML,
        epochs=CONFIG['epochs'],
        batch=CONFIG['batch'],
        imgsz=CONFIG['imgsz'],
        patience=CONFIG['patience'],
        device=CONFIG['device'],
        
        # Augmentation
        augment=True,
        degrees=CONFIG['degrees'],
        translate=CONFIG['translate'],
        scale=CONFIG['scale'],
        fliplr=CONFIG['fliplr'],
        mosaic=CONFIG['mosaic'],
        
        # Optimizer
        optimizer=CONFIG['optimizer'],
        lr0=CONFIG['lr0'],
        lrf=CONFIG['lrf'],
        
        # Project organization
        project=PROJECT_DIR,
        name=RUN_NAME,
        exist_ok=True,
        resume=RESUME_TRAINING,
        
        # Performance settings
        workers=CONFIG['workers'],
        cache=CONFIG['cache'],
        amp=CONFIG['amp'],
        
        # Logging
        verbose=True,
        plots=True,
    )
    
    print("\n" + "=" * 60)
    print("✅ TRAINING COMPLETE!")
    print(f"📁 Best model: {PROJECT_DIR}/{RUN_NAME}/weights/best.pt")
    print("=" * 60)
    
except KeyboardInterrupt:
    print("\n" + "=" * 60)
    print("⏸️  Training interrupted by user (Ctrl+C)")
    print(f"💾 Checkpoint saved: {CHECKPOINT_PATH}")
    print("   To resume: Just run this cell again!")
    print("=" * 60)
    
except RuntimeError as e:
    error_msg = str(e).lower()
    
    if 'cuda' in error_msg and ('illegal' in error_msg or 'error' in error_msg):
        print("\n" + "=" * 60)
        print("❌ CUDA ERROR DETECTED")
        print("=" * 60)
        print(f"Error: {e}\n")
        
        # Check if checkpoint exists
        if os.path.exists(CHECKPOINT_PATH):
            checkpoint_size = os.path.getsize(CHECKPOINT_PATH) / (1024**2)
            print("✅ YOUR PROGRESS IS SAFE!")
            print(f"   Checkpoint: {CHECKPOINT_PATH}")
            print(f"   Size: {checkpoint_size:.1f} MB")
            
            # Check epoch count
            if os.path.exists(f'{PROJECT_DIR}/{RUN_NAME}/results.csv'):
                import pandas as pd
                df = pd.read_csv(f'{PROJECT_DIR}/{RUN_NAME}/results.csv')
                print(f"   Completed epochs: {len(df)}")
                print(f"   Next resume: Epoch {len(df) + 1}")
        
        print("\n🔧 FIX OPTIONS:")
        print("-" * 60)
        print("\n📍 OPTION 1: Try reducing batch size (Quick)")
        print("   In cell above (Section 4), change:")
        print("   CONFIG['batch'] = 8  # Was 16")
        print("   Then re-run this cell")
        
        print("\n📍 OPTION 2: Reinstall PyTorch (Proper fix)")
        print("   Open terminal and run:")
        print("   conda activate pricelens")
        print("   pip uninstall torch torchvision torchaudio -y")
        print("   pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121")
        print("   Then restart Jupyter kernel and re-run this cell")
        
        print("\n📍 OPTION 3: Switch to Google Colab (Easiest)")
        print("   See the next cell below for Colab migration instructions!")
        print("   Your checkpoint will be uploaded and training continues there.")
        
        print("\n" + "=" * 60)
        raise
        
    else:
        # Other error
        print(f"\n❌ Training failed: {e}")
        print(f"Error type: {type(e).__name__}\n")
        
        if os.path.exists(CHECKPOINT_PATH):
            print(f"💾 Checkpoint exists: {CHECKPOINT_PATH}")
            print("   You can resume by running this cell again")
        
        import traceback
        traceback.print_exc()
        raise

except Exception as e:
    print(f"\n❌ Unexpected error: {e}")
    print(f"Error type: {type(e).__name__}\n")
    
    if os.path.exists(CHECKPOINT_PATH):
        print(f"💾 Checkpoint exists: {CHECKPOINT_PATH}")
    
    import traceback
    traceback.print_exc()
    raise

# Update for later cells
RESULTS_DIR = f'{PROJECT_DIR}/{RUN_NAME}'

In [ ]:
# ============================================================
# Colab Checkpoint Restore (Run this in Colab ONLY)
# ============================================================

import sys
import os

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🌐 Google Colab detected!")
    print("=" * 60)
    print("📤 Upload your checkpoint package...")
    print("   Click 'Choose Files' and select: colab_migration_checkpoint.zip")
    print("=" * 60)
    
    from google.colab import files
    import zipfile
    
    # Upload the migration package
    uploaded = files.upload()
    
    if 'colab_migration_checkpoint.zip' in uploaded:
        print("\n✅ Checkpoint package received!")
        print("📦 Extracting...")
        
        # Create checkpoint directory
        PROJECT_DIR = '../models/training_runs'
        RUN_NAME = 'yolo11n_cleveland_notebook'
        CHECKPOINT_DIR = f'{PROJECT_DIR}/{RUN_NAME}'
        
        os.makedirs(f'{CHECKPOINT_DIR}/weights', exist_ok=True)
        
        # Extract checkpoint
        with zipfile.ZipFile('colab_migration_checkpoint.zip', 'r') as zf:
            # Extract last.pt to checkpoint location
            if 'last.pt' in zf.namelist():
                zf.extract('last.pt', f'{CHECKPOINT_DIR}/weights/')
                print("   ✓ Restored checkpoint: last.pt")
            
            # Extract results.csv if present
            if 'results.csv' in zf.namelist():
                zf.extract('results.csv', CHECKPOINT_DIR)
                print("   ✓ Restored training history: results.csv")
                
                # Show progress
                import pandas as pd
                df = pd.read_csv(f'{CHECKPOINT_DIR}/results.csv')
                last_epoch = len(df)
                print(f"\n📈 Restored Progress:")
                print(f"   Completed epochs: {last_epoch}/100")
                print(f"   Will resume from epoch: {last_epoch + 1}")
            
            # Extract best.pt if present
            if 'best.pt' in zf.namelist():
                zf.extract('best.pt', f'{CHECKPOINT_DIR}/weights/')
                print("   ✓ Restored best weights: best.pt")
        
        print("\n" + "=" * 60)
        print("✅ CHECKPOINT RESTORED!")
        print("=" * 60)
        print("\n👉 Now run the training cell above (Section 5A)")
        print("   It will automatically resume from your checkpoint!")
        print("=" * 60)
    else:
        print("\n❌ Checkpoint package not found")
        print("   Please upload: colab_migration_checkpoint.zip")
        
else:
    print("ℹ️  This cell is for Colab only")
    print("   Running locally - skip this cell")
    print("   Your checkpoint is already in the right place!")

In [ ]:
# ============================================================
# Colab Migration Helper - Package Your Checkpoint
# ============================================================

import os
import shutil
import zipfile
from pathlib import Path

PROJECT_DIR = '../models/training_runs'
RUN_NAME = 'yolo11n_cleveland_notebook'
CHECKPOINT_DIR = f'{PROJECT_DIR}/{RUN_NAME}'

print("📦 Creating Colab Migration Package")
print("=" * 60)

# Check if checkpoint exists
checkpoint_path = f'{CHECKPOINT_DIR}/weights/last.pt'

if not os.path.exists(checkpoint_path):
    print("❌ No checkpoint found!")
    print(f"   Expected: {checkpoint_path}")
    print("\n   Nothing to migrate. Start training first.")
else:
    # Get checkpoint info
    checkpoint_size = os.path.getsize(checkpoint_path) / (1024**2)
    print(f"✅ Checkpoint found: {checkpoint_path}")
    print(f"   Size: {checkpoint_size:.1f} MB")
    
    # Check progress
    results_csv = f'{CHECKPOINT_DIR}/results.csv'
    if os.path.exists(results_csv):
        import pandas as pd
        df = pd.read_csv(results_csv)
        last_epoch = len(df)
        latest_metrics = df.iloc[-1]
        
        print(f"\n📈 Current Training Progress:")
        print(f"   Completed epochs: {last_epoch}/100")
        print(f"   Progress: {last_epoch}%")
        
        if 'metrics/mAP50(B)' in df.columns:
            map50 = latest_metrics['metrics/mAP50(B)']
            print(f"   Latest mAP@50: {map50:.4f}")
    
    # Create migration package
    print(f"\n📁 Creating migration ZIP...")
    migration_zip = 'colab_migration_checkpoint.zip'
    
    with zipfile.ZipFile(migration_zip, 'w', zipfile.ZIP_DEFLATED) as zf:
        # Add checkpoint
        zf.write(checkpoint_path, 'last.pt')
        print(f"   ✓ Added checkpoint")
        
        # Add results.csv if exists
        if os.path.exists(results_csv):
            zf.write(results_csv, 'results.csv')
            print(f"   ✓ Added training history")
        
        # Add best weights if they exist
        best_path = f'{CHECKPOINT_DIR}/weights/best.pt'
        if os.path.exists(best_path):
            zf.write(best_path, 'best.pt')
            print(f"   ✓ Added best weights")
    
    zip_size = os.path.getsize(migration_zip) / (1024**2)
    print(f"\n✅ Migration package created!")
    print(f"   File: {migration_zip}")
    print(f"   Size: {zip_size:.1f} MB")
    
    print("\n" + "=" * 60)
    print("📤 NEXT STEPS:")
    print("=" * 60)
    print("\n1️⃣  Upload this notebook to Google Colab:")
    print("   • Go to https://colab.research.google.com")
    print("   • File → Upload notebook")
    print("   • Upload: train_card_detector.ipynb")
    
    print("\n2️⃣  In Colab, upload the migration package:")
    print("   • Run the cell below in Colab")
    print("   • It will prompt you to upload the ZIP file")
    print(f"   • Upload: {migration_zip}")
    
    print("\n3️⃣  Continue training from epoch", last_epoch + 1 if os.path.exists(results_csv) else "?")
    print("   • Just run the training cell!")
    print("   • It will automatically resume")
    
    print("\n" + "=" * 60)
    print(f"📁 Your file is ready: {os.path.abspath(migration_zip)}")
    print("=" * 60)

## Section 5B: Google Colab Migration (If Local GPU Fails)

If the above cell keeps failing with CUDA errors, **switch to Google Colab** and continue from your checkpoint!

### 🎯 Your Training Will Resume from Epoch 8!

The checkpoint system works perfectly across different machines. Here's how to migrate:

---

### Step 1: Prepare Checkpoint Package

Run the cell below to create a migration package with your checkpoint.

In [ ]:
# ============================================================
# Check Training Status and Checkpoints
# ============================================================

import pandas as pd

PROJECT_DIR = '../models/training_runs'
RUN_NAME = 'yolo11n_cleveland_notebook'
CHECKPOINT_PATH = f'{PROJECT_DIR}/{RUN_NAME}/weights/last.pt'
RESULTS_PATH = f'{PROJECT_DIR}/{RUN_NAME}/results.csv'

print("📊 Training Status Check")
print("=" * 60)

if os.path.exists(CHECKPOINT_PATH):
    # Get checkpoint info
    checkpoint_size = os.path.getsize(CHECKPOINT_PATH) / (1024 * 1024)
    checkpoint_time = os.path.getmtime(CHECKPOINT_PATH)
    from datetime import datetime
    checkpoint_date = datetime.fromtimestamp(checkpoint_time).strftime('%Y-%m-%d %H:%M:%S')
    
    print(f"✅ Checkpoint found!")
    print(f"   Path: {CHECKPOINT_PATH}")
    print(f"   Size: {checkpoint_size:.1f} MB")
    print(f"   Last updated: {checkpoint_date}")
    
    # Check results.csv to see progress
    if os.path.exists(RESULTS_PATH):
        df = pd.read_csv(RESULTS_PATH)
        df.columns = df.columns.str.strip()
        
        last_epoch = len(df)
        latest = df.iloc[-1]
        
        print(f"\n📈 Training Progress:")
        print(f"   Completed epochs: {last_epoch}/100")
        print(f"   Progress: {last_epoch}%")
        print(f"   Latest mAP@50: {latest.get('metrics/mAP50(B)', 'N/A'):.4f}")
        print(f"   Latest mAP@50-95: {latest.get('metrics/mAP50-95(B)', 'N/A'):.4f}")
        
        if last_epoch < 100:
            print(f"\n🔄 Next run will resume from epoch {last_epoch + 1}")
        else:
            print(f"\n✅ Training appears complete!")
    else:
        print("\n⚠️  No results.csv found - checkpoint may be from initial setup")
        print("   Will start training from beginning")
else:
    print("❌ No checkpoint found")
    print("   Training will start fresh from epoch 1")
    print("   Using pretrained YOLO11n weights as starting point")

print("=" * 60)

## Section 6: Evaluate Results

Now let's see how well our model trained! We'll look at:

### Training Curves
- **Loss curves**: Should decrease over time (model is learning)
- **Metric curves**: mAP, precision, recall should increase

### What the metrics mean
- **Precision**: Of all boxes the model drew, what % were correct?
- **Recall**: Of all actual cards, what % did the model find?
- **mAP@50**: Average precision at 50% IoU overlap (main metric)
- **mAP@50-95**: Stricter - averaged across IoU 50% to 95%

Good results for card detection:
- mAP@50 > 0.85 = Good
- mAP@50 > 0.90 = Great
- mAP@50 > 0.95 = Excellent


In [ ]:
# ============================================================
# Section 6: Evaluate Results
# ============================================================

from IPython.display import Image, display
import pandas as pd

# Results directory from training
RESULTS_DIR = f'{PROJECT_DIR}/{RUN_NAME}'  # pokemon_detector/yolo11n_cleveland_5k

print("📊 Training Results Analysis")
print("=" * 60)

# Show training curves
print("\n📈 Training Curves:")
curves_path = os.path.join(RESULTS_DIR, 'results.png')
if os.path.exists(curves_path):
    display(Image(filename=curves_path, width=900))
else:
    print(f"   ⚠️  Curves not found at {curves_path}")

# Show confusion matrix if available
print("\n🎯 Confusion Matrix:")
cm_path = os.path.join(RESULTS_DIR, 'confusion_matrix.png')
if os.path.exists(cm_path):
    display(Image(filename=cm_path, width=600))
else:
    print("   ⚠️  Confusion matrix not generated")

# Show sample predictions on validation set
print("\n🖼️  Sample Predictions on Validation Set:")
val_preds = os.path.join(RESULTS_DIR, 'val_batch0_pred.jpg')
if os.path.exists(val_preds):
    display(Image(filename=val_preds, width=800))
else:
    print(f"   ⚠️  Validation predictions not found")

# Print final metrics from CSV
print("\n📋 Final Training Metrics:")
print("-" * 60)
metrics_path = os.path.join(RESULTS_DIR, 'results.csv')

if os.path.exists(metrics_path):
    df = pd.read_csv(metrics_path)
    df.columns = df.columns.str.strip()  # Remove whitespace from column names
    
    # Get the last (best) epoch
    final = df.iloc[-1]
    
    # Metrics to display
    metrics_to_show = [
        ('metrics/precision(B)', 'Precision'),
        ('metrics/recall(B)', 'Recall'),
        ('metrics/mAP50(B)', 'mAP@50'),
        ('metrics/mAP50-95(B)', 'mAP@50-95'),
        ('train/box_loss', 'Final Box Loss'),
        ('train/cls_loss', 'Final Class Loss'),
    ]
    
    for col, name in metrics_to_show:
        if col in final:
            value = final[col]
            # Add emoji indicators
            if 'mAP50(B)' in col:
                if value > 0.95:
                    indicator = "🌟 Excellent!"
                elif value > 0.90:
                    indicator = "✅ Great"
                elif value > 0.85:
                    indicator = "👍 Good"
                else:
                    indicator = "📈 Needs more training"
                print(f"   {name:20s}: {value:.4f}  {indicator}")
            else:
                print(f"   {name:20s}: {value:.4f}")
    
    print("-" * 60)
    print(f"   Total epochs trained: {len(df)}")
else:
    print(f"   ⚠️  Metrics file not found at {metrics_path}")


## Section 7: Test the Model

Let's load our trained model and run it on some test images to see it in action!

The model outputs:
- **Bounding boxes**: Coordinates of detected cards
- **Confidence scores**: How sure the model is (0-1)
- **Class labels**: What class was detected (pokemon_card)


In [ ]:
# ============================================================
# Section 7: Test the Model
# ============================================================

# Load the best trained model
BEST_MODEL_PATH = f'{RESULTS_DIR}/weights/best.pt'

print("🔄 Loading trained model...")
if os.path.exists(BEST_MODEL_PATH):
    best_model = YOLO(BEST_MODEL_PATH)
    print(f"✅ Loaded: {BEST_MODEL_PATH}")
else:
    print(f"❌ Model not found: {BEST_MODEL_PATH}")
    print("   Make sure training completed successfully")
    best_model = None


In [ ]:
# Test on validation images
if best_model and DATASET_PATH:
    # Find validation images (try both 'valid' and 'val' naming)
    val_images_dir = os.path.join(DATASET_PATH, 'valid', 'images')
    if not os.path.exists(val_images_dir):
        val_images_dir = os.path.join(DATASET_PATH, 'val', 'images')
    
    if os.path.exists(val_images_dir):
        all_val_images = [f for f in os.listdir(val_images_dir) 
                         if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        
        # Random sample of 6 images
        test_images = random.sample(all_val_images, min(6, len(all_val_images)))
        
        print(f"🧪 Testing on {len(test_images)} random validation images...\n")
        
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        axes = axes.flatten()
        
        for ax, img_name in zip(axes, test_images):
            img_path = os.path.join(val_images_dir, img_name)
            
            # Run inference
            results = best_model.predict(img_path, conf=0.5, verbose=False)
            
            # Get annotated image
            annotated = results[0].plot()
            annotated = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
            
            # Count detections
            num_detections = len(results[0].boxes)
            
            ax.imshow(annotated)
            ax.set_title(f'{num_detections} card(s) detected', fontsize=10)
            ax.axis('off')
        
        plt.suptitle('Model Predictions on Validation Images (Cleveland 5K)', 
                     fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        # Print detection details for last image
        print("\n📋 Detection details (last image):")
        for i, box in enumerate(results[0].boxes):
            conf = float(box.conf[0])
            cls = int(box.cls[0])
            cls_name = best_model.names[cls]
            bbox = box.xyxy[0].cpu().numpy().astype(int)
            print(f"   Card {i+1}: {cls_name} (conf={conf:.3f}) at [{bbox[0]}, {bbox[1]}, {bbox[2]}, {bbox[3]}]")
    else:
        print(f"❌ Validation images not found")
        print(f"   Checked: {val_images_dir}")


## Section 8: Export for PriceLens

Now let's export our trained model so it can be used with PriceLens!

### What You'll Get
- A `.pt` file containing the trained model weights
- In Colab: Automatic download to your computer
- Locally: File ready in the current directory

### Integration Steps
After downloading the model:
1. Copy the `.pt` file to your PriceLens `models/` directory
2. Update `config.yaml` with the new model path
3. Set `use_card_specific_model: true` (important!)


In [ ]:
# ============================================================
# Section 8: Export for PriceLens
# ============================================================

import shutil

# Export filename - includes dataset info for clarity
EXPORT_NAME = 'pokemon_card_detector_5k.pt'

print("📦 Exporting model for PriceLens...")
print("=" * 60)

if os.path.exists(BEST_MODEL_PATH):
    # Copy to current directory with a clear name
    shutil.copy(BEST_MODEL_PATH, EXPORT_NAME)
    
    # Get model info
    model_size_mb = os.path.getsize(EXPORT_NAME) / (1024 * 1024)
    
    print(f"✅ Model exported: {EXPORT_NAME}")
    print(f"   Size: {model_size_mb:.2f} MB")
    print(f"   Classes: {best_model.names}")
    print(f"   Trained on: Cleveland 5K real-world dataset")
    
    # Download in Colab
    if IN_COLAB:
        print("\n📥 Starting download...")
        from google.colab import files
        files.download(EXPORT_NAME)
        print("   Download started! Check your browser downloads.")
    else:
        export_path = os.path.abspath(EXPORT_NAME)
        print(f"\n📁 Model saved to: {export_path}")
        
        # Also copy to models directory if we're in the project
        models_dir = "../models"
        if os.path.exists(models_dir):
            dest = os.path.join(models_dir, EXPORT_NAME)
            shutil.copy(EXPORT_NAME, dest)
            print(f"   Also copied to: {os.path.abspath(dest)}")
else:
    print(f"❌ Best model not found at {BEST_MODEL_PATH}")
    print("   Make sure training completed successfully")


## Integration with PriceLens

Congratulations! You've trained a Pokemon card detector on **5,058 real-world images**!

### Step 1: Copy the Model
Move `pokemon_card_detector_5k.pt` to your PriceLens `models/` directory:
```bash
cp pokemon_card_detector_5k.pt /path/to/PriceLens/models/
```

### Step 2: Update config.yaml
Edit your `config.yaml` file:
```yaml
detection:
  model_path: "models/pokemon_card_detector_5k.pt"
  use_card_specific_model: true  # IMPORTANT! Disables COCO class filtering
  confidence_threshold: 0.5
```

### Step 3: Test It!
```bash
# From PriceLens directory
python scripts/test_detector_standalone.py
```

Or run the full web app:
```bash
python run_web.py
```

---

## Expected Performance

With the Cleveland 5K dataset, you should see:
- **mAP@50**: 0.85+ (vs ~0.04 with clean scans!)
- **mAP@50-95**: 0.65+
- Robust detection in various lighting
- Handles multiple cards per frame
- Works with cards in hands, on desks, in binders

---

## Troubleshooting

**Model doesn't detect cards well?**
- Lower confidence_threshold to 0.3 in config.yaml
- Check if lighting is too dark/bright
- Ensure cards are visible (not too small in frame)

**Out of memory during training?**
- Reduce batch size: `CONFIG['batch'] = 8` or `4`
- Reduce image size: `CONFIG['imgsz'] = 416`
- Disable caching: `CONFIG['cache'] = False`

**Training is slow?**
- In Colab: Runtime → Change runtime type → GPU (T4)
- Expected: ~1-2 hours for 100 epochs on T4 GPU

---

## Dataset Attribution

Cleveland Pokemon Card Dataset - CC BY 4.0
https://universe.roboflow.com/cleveland-nahux/pokemon-card-identification

---

## What's Next?

Now that detection works, PriceLens can:
1. **Identify** which card it is (using `card_database/` for feature matching)
2. **Look up prices** from APIs
3. **Display** the price overlay

Check out the other notebooks and documentation in the PriceLens repo!
